# Capacitated Vehicle Routing Problem
### by Huanlin Dai

In [2]:
# Code from https://developers.google.com/optimization/routing/cvrp
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import numpy as np
import pandas as pd

In [11]:
def create_data_model():
    """
    Stores the data for the problem.

    Modified:
        1. loads distance matrix from data/distance_matrix_g.csv
        2. loads demands from the 'Daily_Pickup_Totes' column in FUE_Galveston.csv
        3. (temp) num_vehicles defined before vehicle_capacities
        4. (temp) vehicle_capacities made uniform; length of list scaled by num_vehicles
    - Huanlin Dai
    
    """
    fue_data = pd.read_csv("../data/FUE_Galveston.csv")
    data = {}
    data["distance_matrix"] = np.loadtxt('../data/distance_matrix_g.csv', delimiter=',', dtype = int)
    data["demands"] = fue_data['Daily_Pickup_Totes'].astype(int)
    data["num_vehicles"] = 16
    data["vehicle_capacities"] = [15 for _ in range(data["num_vehicles"])]
    data["depot"] = 0
    return data


def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f"Objective: {solution.ObjectiveValue()}")
    total_distance = 0
    total_load = 0
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        plan_output = f"Route for vehicle {vehicle_id}:\n"
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data["demands"][node_index]
            plan_output += f" {node_index} Load({route_load}) -> "
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id
            )
        plan_output += f" {manager.IndexToNode(index)} Load({route_load})\n"
        plan_output += f"Distance of the route: {route_distance}m\n"
        plan_output += f"Load of the route: {route_load}\n"
        print(plan_output)
        total_distance += route_distance
        total_load += route_load
    print(f"Total distance of all routes: {total_distance}m")
    print(f"Total load of all routes: {total_load}")


def main():
    """Solve the CVRP problem."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(
        len(data["distance_matrix"]), data["num_vehicles"], data["depot"]
    )

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data["distance_matrix"][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data["demands"][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data["vehicle_capacities"],  # vehicle maximum capacities
        True,  # start cumul to zero
        "Capacity",
    )

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
    )
    search_parameters.time_limit.FromSeconds(1)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)
    else:
        print("no solution")


if __name__ == "__main__":
    main()

Objective: 331303
Route for vehicle 0:
 0 Load(0) ->  161 Load(1) ->  66 Load(3) ->  99 Load(4) ->  47 Load(5) ->  158 Load(6) ->  4 Load(7) ->  133 Load(9) ->  141 Load(10) ->  45 Load(11) ->  129 Load(12) ->  60 Load(13) ->  149 Load(14) ->  143 Load(15) ->  0 Load(15)
Distance of the route: 79796m
Load of the route: 15

Route for vehicle 1:
 0 Load(0) ->  9 Load(1) ->  11 Load(2) ->  10 Load(3) ->  67 Load(4) ->  152 Load(6) ->  70 Load(7) ->  69 Load(8) ->  64 Load(9) ->  151 Load(10) ->  62 Load(11) ->  61 Load(12) ->  140 Load(13) ->  148 Load(14) ->  38 Load(15) ->  0 Load(15)
Distance of the route: 20534m
Load of the route: 15

Route for vehicle 2:
 0 Load(0) ->  147 Load(3) ->  58 Load(4) ->  56 Load(5) ->  55 Load(6) ->  54 Load(8) ->  8 Load(9) ->  7 Load(10) ->  48 Load(11) ->  100 Load(12) ->  103 Load(13) ->  102 Load(15) ->  0 Load(15)
Distance of the route: 19519m
Load of the route: 15

Route for vehicle 3:
 0 Load(0) ->  88 Load(1) ->  20 Load(2) ->  2 Load(3) ->  27 L